##### TODO Recording

- We're working with same two eventhubs as the previous demo, so no new set up needed
- Both the eventhubs will already have data in them, and we will work with that data
- Make sure you have JoinsWithWatermarks and MoviesRatingsSource(for demo-06) open on different tabs
- We will add some new data as well at the very end

In [0]:
import json

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json, current_timestamp, expr

In [0]:
primaryKey_movies = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath_movies = "EntityPath=loony-movies"

connectionString_movies = primaryKey_movies + ";" + entityPath_movies

ehConf_movies = {}

startingEventPosition_movies = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_movies['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_movies)
ehConf_movies["eventhubs.startingPosition"] = json.dumps(startingEventPosition_movies)

In [0]:
primaryKey_ratings = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath_ratings = "EntityPath=loony-ratings"

connectionString_ratings = primaryKey_ratings + ";" + entityPath_ratings

ehConf_ratings = {}

startingEventPosition_ratings = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_ratings['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_ratings)
ehConf_ratings["eventhubs.startingPosition"] = json.dumps(startingEventPosition_ratings)

In [0]:
streaming_schema_movies = StructType([StructField("name", StringType(), True),
                                      StructField("year", IntegerType(), True),
                                      StructField("director", StringType(), True),
                                      StructField("writer", StringType(), True),
                                      StructField("star", StringType(), True)
                                     ])

streaming_data_movies = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_movies) \
                             .load()

streaming_data_movies = streaming_data_movies.select(from_json(col("body").cast("string"), streaming_schema_movies)) \
                                             .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                             .select(col('data.*'))   

streaming_data_movies.display()

name,year,director,writer,star
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland
Dressed to Kill,1980,Brian De Palma,Brian De Palma,Michael Caine
Somewhere in Time,1980,Jeannot Szwarc,Richard Matheson,Christopher Reeve
Fame,1980,Alan Parker,Christopher Gore,Eddie Barth
The Fog,1980,John Carpenter,John Carpenter,Adrienne Barbeau
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson


In [0]:
streaming_schema_ratings = StructType([StructField("name", StringType(), True),
                                       StructField("rating", StringType(), True),
                                       StructField("score", FloatType(), True)
                                      ])

streaming_data_ratings = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_ratings) \
                             .load()

streaming_data_ratings = streaming_data_ratings.select(from_json(col("body").cast("string"), streaming_schema_ratings)) \
                                               .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                               .select(col('data.*'))   

streaming_data_ratings.display()

name,rating,score
The Shining,R,8.4
The Blue Lagoon,R,5.8
Star Wars: Episode V - The Empire Strikes Back,PG,8.7
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2
Lagaan,PG,9.2
Star Wars: Episode V - The Empire Strikes Back,PG,8.2


In [0]:
streaming_data_movies_timestamp = streaming_data_movies.withColumn("movies_timestamp", current_timestamp())

streaming_data_movies_timestamp.display()

name,year,director,writer,star,movies_timestamp
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine,2021-10-09T06:57:10.741+0000
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood,2021-10-09T06:57:10.741+0000
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau,2021-10-09T06:57:10.741+0000
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams,2021-10-09T06:57:10.741+0000
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland,2021-10-09T06:57:10.741+0000
Dressed to Kill,1980,Brian De Palma,Brian De Palma,Michael Caine,2021-10-09T06:57:10.741+0000
Somewhere in Time,1980,Jeannot Szwarc,Richard Matheson,Christopher Reeve,2021-10-09T06:57:10.741+0000
Fame,1980,Alan Parker,Christopher Gore,Eddie Barth,2021-10-09T06:57:10.741+0000
The Fog,1980,John Carpenter,John Carpenter,Adrienne Barbeau,2021-10-09T06:57:10.741+0000
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,2021-10-09T06:57:10.741+0000


In [0]:
streaming_data_ratings_timestamp = streaming_data_ratings.withColumn("ratings_timestamp", current_timestamp())

streaming_data_ratings_timestamp.display()

name,rating,score,ratings_timestamp
The Shining,R,8.4,2021-10-09T06:57:11.195+0000
The Blue Lagoon,R,5.8,2021-10-09T06:57:11.195+0000
Star Wars: Episode V - The Empire Strikes Back,PG,8.7,2021-10-09T06:57:11.195+0000
Airplane!,PG,7.7,2021-10-09T06:57:11.195+0000
Caddyshack,R,7.3,2021-10-09T06:57:11.195+0000
Friday the 13th,R,6.4,2021-10-09T06:57:11.195+0000
The Blues Brothers,R,7.9,2021-10-09T06:57:11.195+0000
Raging Bull,R,8.2,2021-10-09T06:57:11.195+0000
Lagaan,PG,9.2,2021-10-09T06:57:11.195+0000
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,2021-10-09T06:57:11.195+0000


In [0]:
# Define watermarks

In [0]:
streaming_movies_watermark = streaming_data_movies_timestamp.selectExpr("name as movie_name", "year", "movies_timestamp") \
                                                            .withWatermark("movies_timestamp", "10 seconds") 

streaming_movies_watermark.display()

movie_name,year,movies_timestamp
The Long Riders,1980,2021-10-09T06:57:11.713+0000
Any Which Way You Can,1980,2021-10-09T06:57:11.713+0000
The Gods Must Be Crazy,1980,2021-10-09T06:57:11.713+0000
Popeye,1980,2021-10-09T06:57:11.713+0000
Ordinary People,1980,2021-10-09T06:57:11.713+0000
Dressed to Kill,1980,2021-10-09T06:57:11.713+0000
Somewhere in Time,1980,2021-10-09T06:57:11.713+0000
Fame,1980,2021-10-09T06:57:11.713+0000
The Fog,1980,2021-10-09T06:57:11.713+0000
The Shining,1980,2021-10-09T06:57:11.713+0000


In [0]:
streaming_ratings_watermark = streaming_data_ratings_timestamp.selectExpr("name", "score", "ratings_timestamp") \
                                                              .withWatermark("ratings_timestamp", "20 seconds")

streaming_ratings_watermark.display()

name,score,ratings_timestamp
The Shining,8.4,2021-10-09T06:57:12.198+0000
The Blue Lagoon,5.8,2021-10-09T06:57:12.198+0000
Star Wars: Episode V - The Empire Strikes Back,8.7,2021-10-09T06:57:12.198+0000
Airplane!,7.7,2021-10-09T06:57:12.198+0000
Caddyshack,7.3,2021-10-09T06:57:12.198+0000
Friday the 13th,6.4,2021-10-09T06:57:12.198+0000
The Blues Brothers,7.9,2021-10-09T06:57:12.198+0000
Raging Bull,8.2,2021-10-09T06:57:12.198+0000
Lagaan,9.2,2021-10-09T06:57:12.198+0000
Star Wars: Episode V - The Empire Strikes Back,8.2,2021-10-09T06:57:12.198+0000


In [0]:
watermark_join = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                    expr(""" 
                                                    movie_name = name AND 
                                                    ratings_timestamp >= movies_timestamp AND 
                                                    ratings_timestamp <= movies_timestamp + interval 1 minutes
                                                    """
                                                        )
                                                 )

watermark_join.display()

In [0]:
watermark_join_left_outer = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                                expr(""" 
                                                                movie_name = name AND 
                                                                ratings_timestamp >= movies_timestamp AND 
                                                                ratings_timestamp <= movies_timestamp + interval 2 minutes
                                                                """
                                                                ),
                                                                "leftOuter"
                                                             )

watermark_join_left_outer.display()

In [0]:
watermark_join_left_semi = streaming_movies_watermark.join(streaming_ratings_watermark,
                                                                expr(""" 
                                                                movie_name = name AND 
                                                                ratings_timestamp >= movies_timestamp AND 
                                                                ratings_timestamp <= movies_timestamp + interval 2 minutes
                                                                """
                                                                ),
                                                                "leftSemi"
                                                             )

watermark_join_left_semi.display()

###### TODO Recording

- After running the joins wait for 5 minutes
- Then run the code in demo-06-MoviesRatingsSource which will add to both streams

        event_data_batch.add(EventData('{"name":"Dressed to Kill","year":1980,"director":"Brian De Palma","writer":"Brian De Palma","star":"Michael Caine"}'))
        event_data_batch.add(EventData('{"name":"Somewhere in Time","year":1980,"director":"Jeannot Szwarc","writer":"Richard Matheson","star":"Christopher Reeve"}'))
        event_data_batch.add(EventData('{"name":"Fame","year":1980,"director":"Alan Parker","writer":"Christopher Gore","star":"Eddie Barth"}'))
        event_data_batch.add(EventData('{"name":"The Fog","year":1980,"director":"John Carpenter","writer":"John Carpenter","star":"Adrienne Barbeau"}'))
        
and

        event_data_batch.add(EventData('{ "name": "Fame", "rating": "R", "score": 6.3}'))
        event_data_batch.add(EventData('{ "name": "The Fog","rating": "R", "score": 7.1 }'))
        
        
- Scroll to the join operations of this notebook and show the updates one by one
- After each join result is updated scroll down in the results pane to show the new rows added
- Hopfully in the leftOuter join you will be able to see a row with null padding